# IDL Data

This notebook focuses on handling data in the IDL format.

## Importing the package

In [ ]:
import flekspy

# Set display for XArrays
import xarray as xr

xr.set_options(
    display_expand_coords=False,
    display_expand_data=False,
    display_expand_data_vars=False,
)

## Downloading demo data

If you don't have FLEKS data to start with, you can download demo field data with the following:

In [ ]:
from flekspy.util import download_testfile

url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/batsrus_data.tar.gz"
download_testfile(url, "data")

## Loading data

`flekspy.load` is the interface to read files of all formats. It returns a different object for different formats. IDL format data are processed into XArray data structures:

In [ ]:
file = "data/1d__raw_2_t25.60000_n00000258.out"
ds = flekspy.load(file)
ds

The coordinates can be accessed via

In [ ]:
ds.coords

The variables can be accessed via

In [ ]:
ds.var

and individual variables can be accessed through keys or properties:

In [ ]:
ds["p"]
ds.p

### Unstructured data

Unstructured data is handled with [Xugrid](https://deltares.github.io/xugrid/index.html), with support of regridding and partitioning.

In [ ]:
file = "data/bx0_mhd_6_t00000100_n00000352.out"
dsu = flekspy.load(file)
dsu


### Streamlines and Contours with `FleksAccessor`

The `FleksAccessor` provides convenient methods for adding streamlines and contours, which supports both structured and unstructured data. For unstructured data, `add_stream` automatically interpolates the vector field onto a regular grid.

In [ ]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(8, 6))

# Plot density contour
dsu.fleks.add_contour(ax, "Rho", colors="grey", linewidths=0.5)

# Add streamlines for velocity (Ux, Uy)
dsu.fleks.add_stream(ax, "Ux", "Uy", color="tab:blue", density=1.5)

ax.set_title("Density Contours and Velocity Streamlines (Unstructured)")
plt.show()

### Interpolating data

Variable interpolation is supported via [XArray](https://docs.xarray.dev/en/stable/user-guide/interpolation.html). For 1D data, simply provide the coordinate:

In [ ]:
ds["Rho"].interp(x=100)

For multidimensional interpolation, it can be easily extended as

In [ ]:
file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
ds["rhoS0"].interp(x=-28000.0, y =0.0)

If you want to interpolate for all variables at a single location,

In [ ]:
ds.interp(x=-28000.0, y =0.0)

## Visualizing fields

Thanks to XArray's Matplotlib wrapper, all Matplotlib's plotting functionalities are directly supported.

In [ ]:
import matplotlib.pyplot as plt

file = "data/1d__raw_2_t25.60000_n00000258.out"
ds = flekspy.load(file)

fig, axs = plt.subplots(2, 1, constrained_layout=True, sharex=True, sharey=True)
ds.p.plot(ax=axs[0])
ds.Bx.plot(ax=axs[1])
plt.show()

By default, XArray's 2D plotting functions map the first dimension to the vertical y-axis and the second dimension to the horizontal x-axis. To overwrite this default behavior, we can explicitly set the `x` and `y` arguments:

In [ ]:
file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
fig, axs = plt.subplots(
    3, 1, figsize=(10, 6), constrained_layout=True, sharex=True, sharey=True
)
ds.Bx.plot.pcolormesh(ax=axs[0], x="x", y="y")
ds.By.plot.pcolormesh(ax=axs[1], x="x", y="y")
ds.Bz.plot.pcolormesh(ax=axs[2], x="x", y="y")
plt.show()

Unstructured IDL format output plotting is supported directly by [Xugrid](https://deltares.github.io/xugrid/examples/plotting.html). Note that here we need an additional level `ugrid` compared to direct plotting via `xarray`.

In [ ]:
dsu.Rho.ugrid.plot.contourf()
plt.show()

## Derived Variables

### Pressure Anisotropy

In this demo, the ion pressure anisotropy can be as large as 60 because in a 1D quasi-perpendicular shock simulation, there is no parallel heating.

In [ ]:
import matplotlib.pyplot as plt

file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
anisotropy = ds.derived.get_pressure_anisotropy(species=1)
fig, ax = plt.subplots(1, 1, figsize=(8, 2), constrained_layout=True)
anisotropy.plot.pcolormesh(
    ax=ax,
    x="x",
    y="y",
    cmap="GnBu",
    vmin=0,
    vmax=60,
    cbar_kwargs=dict(label=r"$P_{i\perp}/P_{i\parallel}$", pad=0.0, aspect=25),
)
plt.show()

### Current density

Current density can be calculated in two ways: from the curl of the magnetic field or from the definition $J = \sum_\alpha n_\alpha q_\alpha \vec{v}_\alpha$. Both methods are available in the IDL accessor. As can be seen below, the current density results may not be identical.

In [ ]:
import matplotlib.colors as colors

current = ds.derived.get_current_density()

norm = colors.SymLogNorm(0.1, linscale=0.4, vmin=-0.03, vmax=0.03)
fig, ax = plt.subplots(figsize=(8,2), constrained_layout=True)
current["jy"].plot.pcolormesh(x="x", y="y", cmap="RdBu_r", norm=norm, cbar_kwargs=dict(label=r"$J_y\,[\mu A/m]$", pad=0.0, aspect=40))
ax.set_xlabel(r"x [km]", fontsize=14)
ax.set_ylabel(r"y [km]", fontsize=14)
plt.show()

In [ ]:
current = ds.derived.get_current_density_from_definition(species=[0, 1])

fig, ax = plt.subplots(figsize=(8,2), constrained_layout=True)
current["jy"].plot.pcolormesh(ax=ax, x="x", y="y", cmap="RdBu_r", vmax=0.03, cbar_kwargs=dict(label=r"$J_y\,[\mu A/m]$", pad=0.0, aspect=40))
ax.set_xlabel(r"x [km]", fontsize=14)
ax.set_ylabel(r"y [km]", fontsize=14)
plt.show()